In [1]:
import plotly.express as plt 
import pandas as pd
import os
import numpy as np
import sys 
sys.path.append('./modules/')
from mva import *

In [2]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [3]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

Visits: {} [  1.           2.5        250.00000041  16.25000003   6.25      ]
FOR N {} 1
Delay is 5000.0
Meanwaits for 1 are [  0. 210.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]
Meanwait for 1 is 210.0
Meanwaits for 2 are [0.  2.7 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
Meanwait for 2 is 2.7
Meanwaits for 3 are [ 0. 40.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Meanwait for 3 is 40.0
Meanwaits for 4 are [  0. 180.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]
Meanwait for 4 is 180.0
Station 0 throughput 0.00012539184949484695
Station Delay 0.6269592474742347 utilization, 0.6269592474742347 meanclients
Station 1 throughput 0.0003134796237371174
Stati

In [4]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [5]:
print(meanClients["CPU"])

0     0.000000
1     0.084639
2     0.176878
3     0.276321
4     0.382088
5     0.492741
6     0.606282
7     0.720228
8     0.831797
9     0.938164
10    1.036762
11    1.125557
12    1.203238
13    1.269284
14    1.323915
15    1.367940
16    1.402564
17    1.429190
18    1.449250
19    1.464085
20    1.474874
21    1.482604
22    1.488069
23    1.491886
24    1.494525
25    1.496331
26    1.497557
27    1.498382
28    1.498934
29    1.499301
Name: CPU, dtype: float64


In [6]:


responseTimes = {}
responseTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
responseTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
responseTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
responseTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
responseTimes["N"] = meanWaits["N"]
print(responseTimes)


{'CPU': 0        0.000000
1      675.000000
2      732.131662
3      794.392798
4      861.517004
5      932.909505
6     1007.600472
7     1084.240242
8     1161.154028
9     1236.462960
10    1308.260497
11    1374.814216
12    1434.751032
13    1487.185540
14    1531.766887
15    1568.642694
16    1598.359555
17    1621.730973
18    1639.703551
19    1653.243908
20    1663.257435
21    1670.540066
22    1675.757864
23    1679.446562
24    1682.023285
25    1683.804126
26    1685.023215
27    1685.850644
28    1686.407944
29    1686.780714
Name: CPU, dtype: float64, 'SWAP_IN': 0       0.000000
1     525.000000
2     559.561129
3     595.972811
4     633.834130
5     672.582426
6     711.502385
7     749.761468
8     786.473098
9     820.781277
10    851.952195
11    879.453516
12    903.003398
13    922.579169
14    938.386884
15    950.802975
16    960.304221
17    967.401378
18    972.586829
19    976.300410
20    978.912519
21    980.720623
22    981.954494
23    982.785953
24    

In [7]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png")
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

In [8]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [9]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [10]:
produce_graph_and_save(meanClients,"meanClients","n")

In [11]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [12]:
produce_graph_and_save(responseTimes,"Response_Times","ms",True)

In [13]:
newmat = matrix.astype(np.double)
old = newmat.astype(np.double)
for i in range(50000):
    old = newmat.astype(np.double)
    newmat = old.astype(np.double) @ old.astype(np.double)
    newmat = newmat.round(10) #needed because overflow may occur
    pass
print(newmat)
pi = np.array([0]*len(matrix))
pi[0] = 1
pi = pi@newmat
pi.sum()

[[0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]]


1.0

In [14]:
app = [0]*len(pi)
for i in range(len(pi)):
    app[i] = pi[i]*newmat[i][i]
    pass
app

[1.312749418189456e-05,
 8.2046838636841e-05,
 0.8204683890858013,
 0.003466478944181895,
 0.0005127927414802562]